In [3]:
import sys
sys.path.append('../../python/')  
from simulator.memory import *
from interface.load_parameter import *
import math
from periphery import logicGate
from periphery import constant
print(constant.INV)

load the rng distribution and the weights

In [4]:
filename = '../../fake_config.yaml'
config = load_config(filename)
validate_config(config)


===== Loaded Configuration Parameters =====
{
    "dataset": "CIFAR-10",
    "device_type": "SRAM",
    "distribution_file": "../../../DATA/customized_gaussian_current.csv",
    "frequency": 1000000000.0,
    "mode": "memory",
    "model": "ResNet18",
    "precision_mu": 6,
    "precision_sigma": 2,
    "technology": "65"
}

✅ Config validation passed!


In [10]:
distribution_file = config['distribution_file']
customized_distribution = process_gaussian_data(distribution_file, bins_hist=30)
print("mean of customized distribution: ", customized_distribution.mean())
print("std of customized distribution: ", customized_distribution.std())


mean of customized distribution:  -7.58859641791787e-15
std of customized distribution:  0.9999166631941552


sample from the distribution

In [7]:
#generate the gaussian sample
data = torch.randn(1000, 2)
shape = 200
custom_sample(data, shape)

tensor([-1.3585, -0.0380,  0.1031,  0.0049, -0.2543,  0.9352, -0.1995,  0.9606,
         0.6238,  0.8506, -0.2999,  0.0715,  0.8855, -0.8967,  0.4266,  0.1185,
         0.5838,  0.4387,  0.0305,  0.0434, -0.3610,  1.1004, -0.9382, -0.4676,
         0.3208,  0.3341,  0.0878, -1.1989,  0.4548, -0.9593,  0.2745,  0.7079,
        -0.9849, -0.4861,  0.7918,  0.1248, -0.3244, -0.2355, -0.2002, -1.4073,
        -0.9004, -0.0774, -1.2406, -0.4235, -2.2897,  0.4922,  1.7931, -1.6642,
        -0.2744,  2.0361, -0.0459,  0.8340, -1.8562, -0.9097,  0.9353, -0.4972,
         0.5019, -0.9221,  0.4384, -0.4564, -0.4991, -0.6232, -0.0771,  0.0626,
         0.6117,  1.0858,  1.2521, -1.8467, -0.5664,  0.1744,  0.0471,  0.0567,
        -0.1470, -0.5399, -0.3881,  0.0972, -0.2584, -0.2033,  0.7268, -1.9203,
        -0.7793, -0.1038, -0.3289, -0.5755,  0.1576, -1.2840,  0.4706, -0.4424,
        -0.5758, -0.3552,  0.4313,  0.1138, -0.6552, -0.5260,  0.5185,  0.0222,
        -0.0787, -0.0981, -0.8957,  0.26

In [ ]:
class SubArray:
    def __init__(self, input_parameter, tech, cell, module_lib):
        self.input_parameter = input_parameter
        self.tech = tech
        self.cell = cell
        self.module_lib = module_lib
        self.initialized = False
        self.read_dynamic_energy_array = 0
        self.write_dynamic_energy_array = 0

    def initialize(self, num_row, num_col, unit_wire_res, relax_cell_width=False, relax_cell_height=False):
        self.num_row = num_row
        self.num_col = num_col
        self.unit_wire_res = unit_wire_res

        min_cell_height = self.module_lib.constants.MAX_TRANSISTOR_HEIGHT
        min_cell_width = (self.module_lib.constants.MIN_GAP_BET_GATE_POLY + self.module_lib.constants.POLY_WIDTH) * 2

        if self.cell.mem_cell_type == 'SRAM':
            cell_width = max(self.cell.width_in_feature_size, min_cell_width) if relax_cell_width else self.cell.width_in_feature_size
            cell_height = max(self.cell.height_in_feature_size, min_cell_height) if relax_cell_height else self.cell.height_in_feature_size
            self.length_row = self.num_col * cell_width * self.tech['feature_size']
            self.length_col = self.num_row * cell_height * self.tech['feature_size']
        else:
            raise NotImplementedError("Only SRAM type is implemented in this version.")

        self.cap_row1 = self.length_row * 0.2e-15 / 1e-6
        self.cap_row2 = self.length_row * 0.2e-15 / 1e-6
        self.cap_col = self.length_col * 0.2e-15 / 1e-6
        self.res_row = self.length_row * self.unit_wire_res
        self.res_col = self.length_col * self.unit_wire_res

        res_access = self.module_lib.calculate_on_resistance(
            self.cell.width_access_cmos * self.tech['feature_size'], 'NMOS',
            self.input_parameter['temperature'], self.tech
        )
        cap_access = self.module_lib.calculate_drain_cap(
            self.cell.width_access_cmos * self.tech['feature_size'], 'NMOS',
            self.cell.width_in_feature_size * self.tech['feature_size'], self.tech
        )
        self.cell.cap_sram_cell = (
            cap_access +
            self.module_lib.calculate_drain_cap(self.cell.width_sram_cell_nmos * self.tech['feature_size'], 'NMOS',
                                                self.cell.width_in_feature_size * self.tech['feature_size'], self.tech) +
            self.module_lib.calculate_drain_cap(self.cell.width_sram_cell_pmos * self.tech['feature_size'], 'PMOS',
                                                self.cell.width_in_feature_size * self.tech['feature_size'], self.tech) +
            self.module_lib.calculate_gate_cap(self.cell.width_sram_cell_nmos * self.tech['feature_size'], self.tech) +
            self.module_lib.calculate_gate_cap(self.cell.width_sram_cell_pmos * self.tech['feature_size'], self.tech)
        )

        addr_bits = int(math.ceil(math.log2(self.num_row)))
        self.wl_decoder = self.module_lib.RowDecoder('REGULAR_ROW', addr_bits, False, False, self.tech, self.input_parameter)
        self.sense_amp = self.module_lib.SenseAmp(self.input_parameter, self.tech, self.cell)
        self.sram_write_driver = self.module_lib.WriteDriver(self.input_parameter, self.tech, self.cell)
        self.precharger = self.module_lib.Precharger(self.input_parameter, self.tech, self.cell)

        self.initialized = True
